# Import Libraries & Data

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
df = pd.read_csv('../input/valorant-weapon-stats/valorant-stats.csv')
df

In [ ]:
df.info()

# Check Data

In [ ]:
df.isna().any()

In [ ]:
display(df.columns)
display(df.index)

# What are the Weapon Types available in the game?

In [ ]:
df_weapon_type = df['Weapon Type'].value_counts()
df_weapon_type


In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
plt.pie(df_weapon_type, labels=df_weapon_type.index,pctdistance=0.5, autopct='%1.2f%%', shadow=False)
plt.legend(loc='upper right', title='Weapon Type', shadow=True, bbox_to_anchor=(1.15, 1))
plt.show()

# What is the pricing?

## Based on Weapon

In [ ]:
weapon_price = df[['Name', 'Price']].sort_values(by='Price', ascending=False)
weapon_price

In [ ]:
sns.catplot(kind='bar', data=weapon_price, x='Name', y='Price')
plt.xticks(rotation=90)
plt.show()

## Based on Type

In [ ]:
type_price = df.groupby('Weapon Type')[['Weapon Type', 'Price']].agg('sum').sort_values('Price', ascending=False).reset_index()
type_price

In [ ]:
sns.catplot(kind='bar', data=type_price, x='Weapon Type', y='Price')
plt.xticks(rotation=45)
plt.show()

# Which weapon has the highest fire rate?

## Based on Weapon

In [ ]:
fire_rate = df[['Name', 'Fire Rate']].sort_values(by='Fire Rate', ascending=False)
fire_rate.head()

In [ ]:
sns.catplot(data=fire_rate, y='Name', x='Fire Rate', kind='bar')
plt.show()

## Based on Type

In [ ]:
fire_rate_type = df.groupby('Weapon Type')[['Weapon Type', 'Fire Rate']].agg('mean').sort_values(by='Fire Rate', ascending=False).reset_index()
fire_rate_type

In [ ]:
sns.catplot(kind='bar', data=fire_rate_type, x='Weapon Type', y='Fire Rate')
plt.xticks(rotation=45)
plt.title('Fire Rate on Average Based on Weapon Type')
plt.show()

# How is the Wall Penetration distributed across weapons?

In [ ]:
wall_penet = df[['Name', 'Wall Penetration']].groupby('Wall Penetration').agg('count').reset_index()
wall_penet

In [ ]:
sns.catplot(kind='bar', data=wall_penet, x='Wall Penetration', y='Name')
plt.title('Total weapons for each penetration category')
plt.show()

# Which weapons have a high Magazine Capacity?

In [ ]:
capacity = df[['Name', 'Weapon Type', 'Magazine Capacity']].sort_values('Magazine Capacity', ascending=False)
capacity

In [ ]:
plt.subplots(figsize=(12,7))
sns.barplot(data=capacity, x='Magazine Capacity', y='Name', hue='Weapon Type')
plt.show()

# Weapon Damage

In [ ]:
damage = df[['Name', 'Weapon Type', 'HDMG_0','BDMG_0','LDMG_0','HDMG_1','BDMG_1','LDMG_1','HDMG_2','BDMG_2','LDMG_2']]
damage.columns = ['Name', 'Weapon Type', 'HD Close', 'BD Close','LD Close','HD Mid','BD Mid','LD Mid','HD Far','BD Far','LD Far']
damage.head()

## Close Range Damage

In [ ]:
close_damage = damage[['Name','Weapon Type','HD Close','BD Close','LD Close']]
close_damage.head()

In [ ]:
close_damage = pd.melt(close_damage, id_vars =['Name', 'Weapon Type'], value_vars =['HD Close','BD Close','LD Close']).rename(columns=str.title)
close_damage.head()

In [ ]:
sns.catplot(data=close_damage, x='Name', y='Value', hue='Variable', kind='bar')
plt.xticks(rotation=90)
plt.title('Close range damage')
plt.xlabel('Weapon')
plt.ylabel('Damage')
plt.show()

## Mid Range Damage

In [ ]:
mid_damage = damage[['Name','Weapon Type','HD Mid','BD Mid','LD Mid']]
mid_damage.head()

In [ ]:
mid_damage = pd.melt(mid_damage, id_vars =['Name', 'Weapon Type'], value_vars =['HD Mid','BD Mid','LD Mid']).rename(columns=str.title)
mid_damage.head()

In [ ]:
sns.catplot(data=mid_damage, x='Name', y='Value', hue='Variable', kind='bar')
plt.xticks(rotation=90)
plt.title('Mid range damage')
plt.xlabel('Weapon')
plt.ylabel('Damage')
plt.show()

## Far Range Damage

In [ ]:
far_damage = damage[['Name','Weapon Type','HD Far','BD Far','LD Far']]
far_damage.head()

In [ ]:
far_damage = pd.melt(far_damage, id_vars =['Name', 'Weapon Type'], value_vars =['HD Far','BD Far','LD Far']).rename(columns=str.title)
far_damage.head()

In [ ]:
sns.catplot(data=far_damage, x='Name', y='Value', hue='Variable', kind='bar')
plt.xticks(rotation=90)
plt.title('Far range damage')
plt.xlabel('Weapon')
plt.ylabel('Damage')
plt.show()

# Damage Per Weapon

In [ ]:
damage_per_weapon = pd.melt(damage, id_vars =['Name', 'Weapon Type'], value_vars =['HD Close', 'BD Close','LD Close','HD Mid','BD Mid','LD Mid','HD Far','BD Far','LD Far']).rename(columns=str.title)
damage_per_weapon.head()

In [ ]:
damage_per_weapon['Range'] = damage_per_weapon['Variable'].str.split(' ').str[1]
damage_per_weapon.head()

In [ ]:
damage_per_weapon['Aim'] = damage_per_weapon['Variable'].str.split(' ').str[0]
damage_per_weapon.head()

In [ ]:
g = sns.catplot(data=damage_per_weapon, x='Name', y='Value', hue='Aim', kind='bar', row='Weapon Type', col='Range')
g.set_xticklabels(rotation=90)
plt.show()

# Weapon with high damage for less cost: Weapon Damage Cost (WDC)

`assuming only headshots`

In [ ]:
weapon_damage_cost = df[['Name', 'Price', 'HDMG_0', 'HDMG_1', 'HDMG_2']]
weapon_damage_cost.columns = ['Name', 'Price', 'HD Close', 'HD Mid', 'HD Far']
weapon_damage_cost = pd.melt(weapon_damage_cost, id_vars=['Name', 'Price'], value_vars=['HD Close', 'HD Mid', 'HD Far'])
weapon_damage_cost.head()

In [ ]:
weapon_damage_cost['WDC'] = np.round(weapon_damage_cost['Price'] / weapon_damage_cost['value'], 2)
weapon_damage_cost.head()

In [ ]:
weapon_damage_cost_sorted = weapon_damage_cost.sort_values(by='WDC')
weapon_damage_cost_sorted.head()

In [ ]:
sns.catplot(data=weapon_damage_cost_sorted, x='WDC', y='Name', hue='variable', kind='bar')
plt.show()

# Regression Plot

In [ ]:
weapon_damage_cost.head()

In [ ]:
sns.lmplot(data=weapon_damage_cost, x='value', y='Price', ci=None, hue='variable')
plt.show()